# Relation extraction : prototype of a pipeline using the OpenAI API and Spacy 

Use pip environment : cf. instructions.md



In [1]:
import spacy
import psycopg2
import pandas as pd
from spacy.tokens import Span
from spacy import displacy
from time import strftime, gmtime

In [2]:
from itables import init_notebook_mode, show
import re
from importlib import reload
import json, csv, time

In [3]:
import openai

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import postgresql_functions as pgf
import settings as stt

In [35]:
import IPython

In [ ]:
openai.api_key = ""

In [5]:
# reload(pgf)

In [7]:
### connect to the local database
conn = psycopg2.connect(host="localhost", port = 5432, database="espace_intellectuel", 
                        user="postgres", password=stt.dbw)
#conn

In [7]:
#conn.close()

In [178]:
q1 = """
select pk_mathshistory, "name", url, dates, length(biography) as eff, coreferenced_txt, biography 
from mathshistory.mathshistory m 
where biography ~*  'University'
and pk_mathshistory > 121
order by pk_mathshistory
limit 20;


---where pk_mathshistory < 570
---and pk_mathshistory > 117
--where pk_mathshistory in (81, 82, 103, 117, 129, 133, 143, 146, 159, 186);
"""

In [179]:
result = pgf.sql_explore(q1, conn)
# print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

In [180]:
textes = pd.DataFrame(result[0])
textes.columns = ['id', 'name', 'url', 'dates', 'length_bio', 'coref_texte', 'biog']

In [204]:
textes.head(20)

,id,name,url,dates,length_bio,coref_texte,biog
0,122,Giovanni Magini,https://mathshistory.st-andrews.ac.uk/Biograph...,1555-1617,11224,"Giovanni Magini's father, Pasquale Magini, was...","Giovanni Magini's father, Pasquale Magini, was..."
1,124,Thomas Harriot,https://mathshistory.st-andrews.ac.uk/Biograph...,1560-1621,19509,Thomas Harriot was a mathematician and astrono...,Thomas Harriot was a mathematician and astrono...
2,125,Adriaan van Roomen,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1615,13588,Adriaan van Roomen is often known by Roomen La...,Adriaan van Roomen is often known by his Latin...
3,126,Bartholomeo Pitiscus,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1613,5507,Bartholomeo Pitiscus was born into a poor fami...,Bartholomeo Pitiscus was born into a poor fami...
4,127,Duncan Liddel,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1613,12492,Duncan Liddel was the son of John Liddel who i...,Duncan Liddel was the son of John Liddel who i...
5,128,Henry Briggs,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1630,12752,Henry Briggs was the Briggs most responsible f...,Henry Briggs was the man most responsible for ...
6,129,Philip van Lansberge,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1632,16365,Philip van Lansberge's name is given in many d...,Philip van Lansberge's name is given in many d...
7,130,Thomas Fincke,https://mathshistory.st-andrews.ac.uk/Biograph...,1561-1656,12105,Thomas Fincke's name appears with a variety of...,Thomas Fincke's name appears with a variety of...
8,132,Galileo Galilei,https://mathshistory.st-andrews.ac.uk/Biograph...,1564-1642,25070,Galileo Galilei's parents were Vincenzo Galile...,Galileo Galilei's parents were Vincenzo Galile...
9,133,Giuseppe Biancani,https://mathshistory.st-andrews.ac.uk/Biograph...,1566-1624,10035,Giuseppe Biancani's name also appears in name ...,Giuseppe Biancani's name also appears in its L...


In [211]:
### Choose one document
txt = textes.iloc[0].coref_texte  #[:1600]
print(txt)

Giovanni Magini's father, Pasquale Magini, was a citizen of Padua. Nothing else appears to be known concerning Magini father or the rest of Magini family. Magini completed citizen education at the University of Bologna, receiving a doctorate in philosophy in 1579. citizen published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582 and in the following year citizen published an Italian version of the original Latin text.

Egnatio Danti held the chair of mathematics at the University of Bologna and, following citizen death in October 1586, a competition was announced to fill the chair . Magini competed for the chair as did, among others, Galileo. One might think that there would be no competition once Galileo applied for a chair but Magini was nine years older than Galileo and seen as a much safer appointment; citizen was appointed in 1588 and held the position for the rest of citizen life. Galileo had revolutionary ideas while Magini 's ideas were original but based 

## Extract information in form of sentences and prototype a pipeline

In [14]:
text = f"""Michael Mästlin was born in Göppingen which was a village about 50 km east of Tübingen. Mästlin father, Jakob Mästlin, and Mästlin mother, Dorothea Simon, were both devout Lutherans and Michael was brought up in that faith and remained strongly committed to father throughout Mästlin life. Mästlin was the middle child of the family, having an older sister and a younger brother. Mästlin attended the monastic school in Königsbronn then, after Mästlin studies there, entered Tübingen University in 1568. [3]:-
As was the case with many young scholars including Kepler, Mästlin most famous student, [Mästlin ] did Mästlin undergraduate studies at a preparatory school and came to the university to take Mästlin final exams and pick up Mästlin baccalaureate degree.At Tübingen University Mästlin studied mathematics and astronomy for a Master's degree under Philipp Apian who was Peter Apian's son. In 1570, while a student, Mästlin purchased a copy of Copernicus's De revolutionibus from the widow of Victorin Strigel, who had been professor of theology at Leipzig and the author of an astronomy text. Mästlin was awarded a Master 's degree with distinction in 1571. While studying for this degree Mästlin had edited a new edition of the Prutenis Tables which was published in 1571. The Prutenis Tables had been originally compiled by Erasmus Reinhold who had based 1571 on Copernicus's version of the solar system. After graduating, Mästlin spent the following years teaching in Tübingen, and at the same time took [3]:-
... the theological programme, because Tübingen University's primary mission was to prepare young men for the Lutheran ministry.Then, in 1576, Mästlin was sent to be a deacon at the Lutheran Church in Backnang, a town about 30 km northwest of Göppingen. Mästlin married Margarete Gruuninger (1551-1588) in April 1577 not long after Mästlin took up the position in Backnang ; 1551 had three sons and three daughters but Margarete died due to complications with the birth of 1551 sixth child. Mästlin then married Margarete Burkhardt, daughter of a professor at Tübingen; 1551 had eight children.
Mästlin is famous for Mästlin excellent, very accurate, observations of the comet of 1577, observed while Mästlin was in Backnang, and published in Tübingen in 1578 as Observatio et demonstratio cometae aetherae qui anno 1577 et 1578 constitutus in sphaera Veneris apparuit cum admirandius eius passionibus varietate scilicet motus loco orbe distantia a terro centro etc. adhibitis demonstrationibus geometricis et calculo arithmetico cuius modi de alio quoquam cometa nunquam visa est (Observations and demonstrations of the ethereal comets of the years 1577 and 1578). We discuss this achievement in more detail below. Mästlin remained in Backnang for four years, then was appointed as professor of mathematics at the University of Heidelberg in 1580. There Mästlin published the first edition of Mästlin famous astronomy textbook Epitome astronomiae (1582) - Mästlin published six further editions of this popular work during Mästlin lifetime. Despite Mästlin commitment to the views of Copernicus (which we state below in Mästlin own words) this teaching textbook was written purely as a description of astronomy based on the geocentric model of Ptolemy.
After four years in Heidelberg Mästlin returned to Mästlin position in Tübingen where Mästlin spent the rest of Mästlin career [2]:-
At Tübingen , Mästlin was elected dean of the arts faculty several times. Mästlin was well liked by both Mästlin colleagues and Mästlin students. Mästlin was very generous both to Mästlin family and to others. Mästlin was a religious man; Mästlin followed the Lutheran line in opposing the Gregorian calendar reform partly because man was initiated by the Pope. Mästlin had several students who became noted mathematicians, the most famous being Kepler. Mästlin also maintained interests in Biblical chronology and geography.Perhaps Mästlin greatest achievement (other than being Kepler 's teacher) is that Mästlin was the first to compute the orbit of a comet, although Mästlin method was not sound. Mästlin found, however, a sun centred orbit for the comet of 1577 which Mästlin claimed supported Copernicus's heliocentric system. Mästlin did show that the comet was further away than the moon, which contradicted the accepted teachings of Aristotle. Although clearly believing in the system as proposed by Copernicus , Mästlin taught astronomy using Mästlin own textbook which was based on Ptolemy's system. However for the more advanced lectures Mästlin adopted the heliocentric approach - Kepler credited Mästlin with introducing Mästlin to Copernican ideas while Mästlin was a student at Tübingen (1589-94). In [6] Methuen looks at these two different world systems in Mästlin 's teaching. A G Molland reviews Methuen 's paper:-
Michael Mästlin is regularly ascribed a firm if minor place in accounts of the Astronomical Revolution, as the teacher of Johannes Kepler and as an early believer in the physical reality of the Copernican system. What has been less clear is how this related to Kepler regular teaching at the University of Tübingen, with some having argued that Kepler only taught Copernicanism cautiously and in private. Methuen examines this question in the light of Maestlin's published writings, including five sets of theses over whose disputations (two at Heidelberg, one at Tübingen ) Kepler presided. believer shows that Kepler had no compunction about firmly asserting the supralunar position of the New Star of 1572 and of the comets of 1577 - 1578 and 1580. In general, Kepler writings, especially Kepler textbook 'Epitome astronomiae' of 1582, show no firm Copernican commitment, although there are signs of a leaning in that direction, and Kepler was concerned with emphasizing the interconvertibility of hypotheses. Kepler elementary teaching was certainly based on traditional astronomy, but Methuen concludes that Kepler taught newer material to Kepler more advanced students. Methuen claims that Kepler made a clear distinction between "spheres" and "orbs", with the former possessing physical reality and the latter being merely useful mathematical constructions ...Mästlin was both a great expert on spherical trigonometry and also a fine observer producing accurate data - the quality of Mästlin eyesight is seen from the fact that Mästlin saw, and sketched the positions of, 11 stars in the Pleiades cluster. Of course there were not any bright city lights around then, but we challenge any reader to equal this achievement however dark a site reader find. Mästlin seems to have been the first to claim that the dark part of the moon shone through sunlight reflected from the earth but Leonardo da Vinci has also been credited with this idea. Another first for Mästlin is an accurate calculation of the golden ratio as "approximately 0.6180340" stated in a letter Mästlin wrote to Kepler in 1597.
Mästlin was an innovative thinker who was quite prepared to challenge conventional views. For example Mästlin attempted to measure the parallax of a supernova and, having failed to find any, deduced that supernova was as far away as the "fixed stars". This of course contradicted the view, held since Aristotle, that all changes in the heavens occurred closer to earth than the realm of the stars which was unchanging.
Mästlin lived to see the invention of the telescope for astronomical observations by Galileo. Mästlin had two, rather poor, telescopes with which Mästlin was able to observe sunspots and the moons of Jupiter. Mästlin was still making accurate observations at the age of 78 when Mästlin observed the lunar eclipse of 1628.
As we mentioned above Mästlin acquired a copy of Copernicus's De revolutionibus in 1570 and Mästlin wrote extensive notes near the beginning of the book. These give much insight into Mästlin views on Copernicus and we quote the notes using Gingerich's translations. Mästlin wrote that [3]:-
... the arrangement presented in this book is the sort of structure in which all the sidereal motions and phenomena are explained very exactly. Therefore this hypothesis recommends hypothesis to the intellect.Mästlin continued:-
The heavenly motions were at the point of collapse, and so [Copernicus] concluded that appropriate hypotheses were needed to explain these motions . When Mästlin noticed that the common hypotheses were insufficient, Mästlin eventually accepted the idea of the Earth's mobility, since indeed, idea not only satisfied the phenomena very well but idea didn't lead to anything absurd.
In fact, if anyone would straighten out the common hypotheses so that hypotheses would agree with the phenomena and allow no inconsistencies, then I would gratefully trust Mästlin ; clearly Mästlin would bring very many to Mästlin views. But I see that some, even very outstanding mathematicians, have laboured on this yet, in the end, without results. Therefore, I think that unless the common hypotheses are reformed (a task that I am not up to because of my inadequate abilities), I will accept the hypotheses and opinion of Copernicus - after Ptolemy, the prince of all Astronomers.There are further annotations written by Mästlin [3] which are very interesting. On the back of the title page of De revolutionibus is the infamous notice which states that "... these hypotheses need not be true nor even probable; it is sufficient if the calculations agree with the observations." Mästlin adds a note to Mästlin copy stating:- 
This preface was added by someone, whoever preface author may be, (for indeed, preface weakness of style and choice of words reveal that preface is not by Copernicus).He later added a further note:-
I found the following words written somewhere among the books of Philipp Apian (which I bought from Apian widow); although no author was given I could recognise Apian 's hand:
On account of this letter Georg Joachim Rheticus, the Leipzig professor and disciple of Copernicus, became involved in a very bitter wrangle with the printer, who asserted that wrangle had been turned over to Rheticus with the rest of the work. Rheticus , however, suspected that Osiander [the proof-reader of Copernicus 's book] had prefaced Copernicus to the work . If professor knew this for certain, professor declared, professor would handle that fellow so that in future professor would mind professor own business and not slander astronomers any more. Nevertheless, [Peter] Apian told me that Osiander had openly admitted to professor that professor had added this all by professor .
"""


In [22]:
messages = []

# optionnel, permet de définir le comportement que l'assistant doit adopter
messages.append({"role": "system", "content": "You are a NLP application to extract named entities and relations among them from texts"})

# Une question classique qu'on pourrait poser à ChatGPT
messages.append({"role": "user", "content": "Extract all studying and learning activities from following text as structured data"})

messages.append({"role": "assistant", "content": text})


# messages.append({"role": "user", "content": "get all study activities like Kepler studied mathematics at the University of Graz from 1590 and 1598"})

messages.append({"role": "user", "content": "produce a list of study activities in this form: Michael Mästlin studied mathematics and astronomy at Tübingen University - 1568 to 1571."})

messages.append({"role": "user", "content": "produce a list of study activities in this form: Michael Mästlin studied mathematics and astronomy at Tübingen University - 1568 to 1571."})


In [24]:
#messages

In [25]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo", 
  temperature=0.2,
  messages=messages #[{"role": "user", "content": prompt}]
)

In [26]:
print(response['choices'][0]['message']['content'])

- Michael Mästlin studied mathematics and astronomy at Tübingen University from 1568 to 1571.


In [98]:
my_openai_obj = list(response.choices)[0]
oai_txt = (my_openai_obj.to_dict()['message']['content'])

In [99]:
oai_txt.split('\n')
ls

['Michael Mästlin, was born in Göppingen, around 50 km east of Tübingen. ',
 'Mästlin attended the monastic school in Königsbronn and entered Tübingen University in 1568. ',
 "Mästlin studied mathematics and astronomy for a Master's degree under Philipp Apian at Tübingen University and was awarded a Master's degree with distinction in 1571. ",
 'Mästlin spent the following years teaching in Tübingen and at the same time took the theological program. ',
 'In 1576, Mästlin was sent to be a deacon at the Lutheran Church in Backnang. ',
 'Mästlin married Margarete Gruuninger in April 1577 and had three sons and three daughters. ',
 'Mästlin was appointed as a professor of mathematics at the University of Heidelberg in 1580. ',
 'Mästlin returned to Mästlin position in Tübingen after four years in Heidelberg. ',
 'Mästlin was elected dean of the arts faculty several times at Tübingen. ',
 'Mästlin was the first to compute the orbit of a comet in 1577. ',
 'Mästlin published the first editio

In [ ]:
messages.append({"role": "user", "content": "add into json output the original sentence from which you extract the activity of the person"})
        

## EntityLinker

* https://spacy.io/api/entitylinker


In [69]:
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("entityLinker", last=True)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/francesco/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
### Create a Spacy model for the extracted sentences
doc = nlp(oai_txt)

In [73]:
all_linked_entities = doc._.linkedEntities
for sent in doc.sents:
    sent._.linkedEntities.pretty_print()

<EntityElement: https://www.wikidata.org/wiki/Q75797 Michael Maestlin          German astronomer and mathematician               >
<EntityElement: https://www.wikidata.org/wiki/Q4072 Göppingen                 town in Göppingen District, Stuttgart, Baden-Württemberg, Germany>
<EntityElement: https://www.wikidata.org/wiki/Q970 Comoros                   sovereign archipelago island nation in the Indian Ocean>
<EntityElement: https://www.wikidata.org/wiki/Q3806 Tübingen                  town in central Baden-Württemberg, Germany        >
<EntityElement: https://www.wikidata.org/wiki/Q1640151 Monastic school                                                             >
<EntityElement: https://www.wikidata.org/wiki/Q1796326 Königsbronn               municipality in Germany                           >
<EntityElement: https://www.wikidata.org/wiki/Q3806 Tübingen                  town in central Baden-Württemberg, Germany        >
<EntityElement: https://www.wikidata.org/wiki/Q395 mathematics  

### Explore Spacy model

In [74]:
info = []
for token in doc:
    info.append((token.idx, token.ent_id_,token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
                token.shape_, token.is_alpha, token.is_stop, token.tag_, token.vector[:1]))
tx_df  = pd.DataFrame(info, columns = ["idx", "ent_id_","text", "lemma_", "pos_", "tag_", "dep_","shape_", "is_alpha", "is_stop", "tag_", "vector"])

In [75]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(tx_df[tx_df.idx >= 0], classes="display", scrollY="400px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

## Relation extraction

In [76]:
### https://towardsdatascience.com/nlp-with-python-knowledge-graph-12b93146a458

def extract_entities(doc):
    a, b, prev_dep, prev_txt, prefix, modifier = "", "", "", "", "", ""
    for token in doc:
        if token.dep_ != "punct":
            ## prexif --> prev_compound + compound
            if token.dep_ == "compound":
                prefix = prev_txt +" "+ token.text if prev_dep == "compound" else token.text
            
            ## modifier --> prev_compound + %mod
            if token.dep_.endswith("mod") == True:
                modifier = prev_txt +" "+ token.text if prev_dep == "compound" else token.text
            
            ## subject --> modifier + prefix + %subj
            if token.dep_.find("subj") == True:
                a = modifier +" "+ prefix + " "+ token.text
                prefix, modifier, prev_dep, prev_txt = "", "", "", ""
            
            ## if object --> modifier + prefix + %obj
            if token.dep_.find("obj") == True:
                b = modifier +" "+ prefix +" "+ token.text
            
            prev_dep, prev_txt = token.dep_, token.text
    
    # clean
    a = " ".join([i for i in a.split()])
    b = " ".join([i for i in b.split()])
    return (a.strip(), b.strip())


# The relation extraction requires the rule-based matching tool, 
# an improved version of regular expressions on raw text.
def extract_relation(doc, nlp):
    matcher = spacy.matcher.Matcher(nlp.vocab)
    p1 = [{'DEP':'ROOT'}, 
          {'DEP':'prep', 'OP':"?"},
          {'DEP':'agent', 'OP':"?"},
          {'POS':'ADJ', 'OP':"?"}] 
    matcher.add(key="matching_1", patterns=[p1]) 
    matches = matcher(doc)
    k = len(matches) - 1
    span = doc[matches[k][1]:matches[k][2]] 
    return span.text

In [84]:
sents = [s for s in doc.sents]
sents[2]

Mästlin attended the monastic school in Königsbronn and entered Tübingen University in 1568.

In [85]:
st = sents[2]

In [86]:
info = []
for token in st:
    info.append((token.text, token.lemma_, token.ent_kb_id_, token.ent_type_, token.pos_, token.tag_, token.dep_,
                token.shape_, token.is_alpha, token.is_stop))
tx_df  = pd.DataFrame(info, columns = ["text", "lemma_", "kb_id", "ent_type", "pos_", "tag_", "dep_","shape_", "is_alpha", "is_stop"])

In [87]:
### https://github.com/mwouts/itables/blob/main/docs/advanced_parameters.md
show(tx_df, classes="display", scrollY="200px", scrollCollapse=True, paging=False, column_filters="footer", dom="lrtip")

In [88]:
extract_entities(st)

('Mästlin', 'monastic Tübingen 1568')

In [89]:
extract_relation(st, nlp)

'attended'

## Extract information in form of triples using OpenAI API

In [205]:
# Specify the file name and mode ('w' for writing)
filename = "data/openai_test_result_3.csv"

In [236]:
header = ['id_mathshistory','response']
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(header)

In [237]:
%%time

results = []

# Specify the file name and mode ('w' for writing)
filename = filename


i = 0

for t in textes.values:
    
    if i >= 0:
    
        messages = []

        # optionnel, permet de définir le comportement que l'assistant doit adopter
        messages.append({"role": "system", "content": """You are a NLP application to extract named entities and relations 
        among them from texts in form of structured data"""})

        # Une question classique qu'on pourrait poser à ChatGPT
        messages.append({"role": "user", "content": f"""Extract all studying and learning activities from following text:
                         
                         TEXT
                         {t[6][:3500]}"""})

        
        messages.append({"role": "user", "content": """produce a list of studying activities according to following example: 
                         Pinco Pallino studied mathematics and astronomy at Ardauschi University - from 1568 to 1571."""})
        
        messages.append({"role": "user", "content": "output in json format"})
        
        messages.append({"role": "user", "content": "add into json output the original sentence from which you extracted the study activity of the person"})
        
        
        messages.append({"role": "assistant", "content":"""{"person": "Pinco Pallino", 
             "activity": "studied", "object": "mathematics and astronomy", 
             "institution": "Sapienza University",
             "begin": 1568, "end": 1571,
             "original_sentence": "In Rome Pinco Pallino studied mathematics and astronomy at the Sapienza University "}"""})

        messages.append({"role": "user", "content": "the example that you provided is to be applied"})

        response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo", 
              temperature=0,
              messages=messages #[{"role": "user", "content": prompt}]
            )

        r = response['choices'][0]['message']['content']
        
        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file, delimiter='\t')
            writer.writerow([t[0], r])

        results.append([t[0], r])
        
        time.sleep(22)
        
    i =+ 1     

CPU times: user 180 ms, sys: 11.8 ms, total: 192 ms
Wall time: 9min 57s


In [261]:
rslt = results[0][1]
print(rslt)

{"person": "Giovanni Magini", 
             "activity": "completed his education", "object": "at the University of Bologna", 
             "begin": 1579, "end": 1579,
             "original_sentence": "Magini completed his education at the University of Bologna, receiving a doctorate in philosophy in 1579."}

{"person": "Giovanni Magini", 
             "activity": "published", "object": "Ephemerides coelestium motuum", 
             "begin": 1582, "end": 1582,
             "original_sentence": "He published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582."}

{"person": "Giovanni Magini", 
             "activity": "published", "object": "an Italian version of the original Latin text", 
             "begin": 1583, "end": 1583,
             "original_sentence": "and in the following year he published an Italian version of the original Latin text."}

{"person": "Giovanni Magini", 
             "activity": "competed", "object": "for the chair", 
             "begin": n

In [264]:
results[0]

[122,
 '{"person": "Giovanni Magini", \n             "activity": "completed his education", "object": "at the University of Bologna", \n             "begin": 1579, "end": 1579,\n             "original_sentence": "Magini completed his education at the University of Bologna, receiving a doctorate in philosophy in 1579."}\n\n{"person": "Giovanni Magini", \n             "activity": "published", "object": "Ephemerides coelestium motuum", \n             "begin": 1582, "end": 1582,\n             "original_sentence": "He published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582."}\n\n{"person": "Giovanni Magini", \n             "activity": "published", "object": "an Italian version of the original Latin text", \n             "begin": 1583, "end": 1583,\n             "original_sentence": "and in the following year he published an Italian version of the original Latin text."}\n\n{"person": "Giovanni Magini", \n             "activity": "competed", "object": "for the chair", 

### Inspect the produced data

In [ ]:
json_data = []
with open(filename, newline='') as f:    
    my_reader = csv.reader(f, delimiter='\t')
    for row in my_reader:
        # print(row[1])
        json_data.append(row[1])

In [ ]:
json_data[1]

In [267]:
rj = []
for r in results:
    print(r[0])
    try:
        rr = r[1].replace('\n', '').replace('  ', '').replace('}{', '}|{').split('|')
        print(rr)
        rp = [json.loads(e) for e in rr]
        rj.append([r[0], rp])
    except Exception as e:
        print('\n\nDoc:'+ str(r[0]) + ' — ' + str(e) + '\n\n')

122
['{"person": "Giovanni Magini","activity": "completed his education", "object": "at the University of Bologna","begin": 1579, "end": 1579, "original_sentence": "Magini completed his education at the University of Bologna, receiving a doctorate in philosophy in 1579."}', '{"person": "Giovanni Magini","activity": "published", "object": "Ephemerides coelestium motuum","begin": 1582, "end": 1582, "original_sentence": "He published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582."}', '{"person": "Giovanni Magini","activity": "published", "object": "an Italian version of the original Latin text","begin": 1583, "end": 1583, "original_sentence": "and in the following year he published an Italian version of the original Latin text."}', '{"person": "Giovanni Magini","activity": "competed", "object": "for the chair","begin": null, "end": null, "original_sentence": "Magini competed for the chair as did, among others, Galileo."}', '{"person": "Giovanni Magini","activity": 

In [285]:
rj[7]

[130,
 [{'person': 'Thomas Fincke',
   'activity': 'studied',
   'object': 'mathematics and astrology, philosophy and rhetoric',
   'institution': 'Strasbourg Academy',
   'begin': 1577,
   'end': 1582,
   'original_sentence': 'He spent five years studying mathematics and astrology, philosophy and rhetoric at Strasbourg.'}]]

In [287]:
rj[10]

[135,
 [{'person': 'Johannes Kepler',
   'activity': 'discovering the three laws of planetary motion',
   'institution': 'N/A',
   'begin': 1609,
   'end': 1619,
   'original_sentence': 'Johannes Kepler is now chiefly remembered for discovering the three laws of planetary motion that bear his name published in 1609 and 1619)'},
  {'person': 'Johannes Kepler',
   'activity': 'work in optics',
   'institution': 'N/A',
   'begin': 1604,
   'end': 1611,
   'original_sentence': 'He also did important work in optics (1604, 1611)'},
  {'person': 'Johannes Kepler',
   'activity': 'discovered two new regular polyhedra',
   'institution': 'N/A',
   'begin': 1619,
   'end': 1619,
   'original_sentence': 'discovered two new regular polyhedra (1619)'},
  {'person': 'Johannes Kepler',
   'activity': 'gave the first mathematical treatment of close packing of equal spheres',
   'institution': 'N/A',
   'begin': 1611,
   'end': 1611,
   'original_sentence': 'gave the first mathematical treatment of clo

In [288]:
rj[0]

[122,
 [{'person': 'Giovanni Magini',
   'activity': 'completed his education',
   'object': 'at the University of Bologna',
   'begin': 1579,
   'end': 1579,
   'original_sentence': 'Magini completed his education at the University of Bologna, receiving a doctorate in philosophy in 1579.'},
  {'person': 'Giovanni Magini',
   'activity': 'published',
   'object': 'Ephemerides coelestium motuum',
   'begin': 1582,
   'end': 1582,
   'original_sentence': 'He published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582.'},
  {'person': 'Giovanni Magini',
   'activity': 'published',
   'object': 'an Italian version of the original Latin text',
   'begin': 1583,
   'end': 1583,
   'original_sentence': 'and in the following year he published an Italian version of the original Latin text.'},
  {'person': 'Giovanni Magini',
   'activity': 'competed',
   'object': 'for the chair',
   'begin': None,
   'end': None,
   'original_sentence': 'Magini competed for the chair as did,

In [271]:
jf = "data/openai_test_result_3.json"
with open(jf, "w") as f:
    f.write(json.dumps(rj))

In [274]:
with open(jf) as f:
    loaded_file = json.load(f)

In [286]:
loaded_file[0][1]

[{'person': 'Giovanni Magini',
  'activity': 'completed his education',
  'object': 'at the University of Bologna',
  'begin': 1579,
  'end': 1579,
  'original_sentence': 'Magini completed his education at the University of Bologna, receiving a doctorate in philosophy in 1579.'},
 {'person': 'Giovanni Magini',
  'activity': 'published',
  'object': 'Ephemerides coelestium motuum',
  'begin': 1582,
  'end': 1582,
  'original_sentence': 'He published Ephemerides coelestium motuum, a major treatise on astronomy, in 1582.'},
 {'person': 'Giovanni Magini',
  'activity': 'published',
  'object': 'an Italian version of the original Latin text',
  'begin': 1583,
  'end': 1583,
  'original_sentence': 'and in the following year he published an Italian version of the original Latin text.'},
 {'person': 'Giovanni Magini',
  'activity': 'competed',
  'object': 'for the chair',
  'begin': None,
  'end': None,
  'original_sentence': 'Magini competed for the chair as did, among others, Galileo.'},
 {'